This notebook is used for Capstone Project

In [1]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [ ]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 

from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize
import folium 

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = '0W1MHKERIP4CAUL5LAVS4XORC2C2D51SB2TGJAHZ44BVICVD' 
CLIENT_SECRET = 'E1HWUVNWVVP4DXCT02QTFF0YNNIAA1M2SWU3QR5JNC1JEOE1'
VERSION = '20180604'
LIMIT=50

In [3]:
address = input('please type in your address:').strip()

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
search_query = input('What type of restaurant are you intereted in? :').strip()
radius =2000  #meters
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)


please type in your address: 1235 mission st, San Francisco


37.7724921 -122.4188648


What type of restaurant are you intereted in? : Japanese


Japanese .... OK!


localize restaurants of interest near the desired location and create a dataframe of the search result

In [7]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# Define information and filter the dataframe: keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dfc=dataframe_filtered[['name','categories','address','distance','lat','lng','id']]
dfc.head()

,name,categories,address,distance,lat,lng,id
0,Ryoko's Japanese Restaurant & Bar,Sushi Restaurant,619 Taylor St,1851,37.788183,-122.411882,433c8000f964a52043281fe3
1,Cha-Ya Vegetarian Japanese Restaurant,Vegetarian / Vegan Restaurant,762 Valencia St,1324,37.760786,-122.421557,44ccee85f964a52016361fe3
2,Muracci's Japanese Curry & Grill,Japanese Curry Restaurant,307 Kearny St,2427,37.791000,-122.404282,49ba1813f964a52059531fe3
3,Japanese YWCA Building / Nihonmachi Little Fri...,Nursery School,1700 Sutter St,1918,37.787017,-122.430609,4d2ca3774ab78eec71103326
4,Miyabi Japanese Restaurant,Sushi Restaurant,253 Church St,1085,37.766829,-122.428903,43a8315cf964a520742c1fe3


import csv form with SF health inspection restaurant_scores (file downloaded from Kaggle)

In [8]:
dfs=pd.read_csv('Restaurant_Scores_-_LIVES_Standard.csv')
dfs.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,835,Kam Po Kitchen,801 Broadway St,San Francisco,CA,94133,37.797223,-122.410513,"(37.797223, -122.410513)",NaN,835_20180917,09/17/2018 12:00:00 AM,88.0,Routine - Unscheduled,835_20180917_103154,Unclean or degraded floors walls or ceilings,Low Risk
1,67448,Soo Fong Restaurant,3801 03rd St 370,San Francisco,CA,94124,37.741829,-122.388098,"(37.741829, -122.388098)",NaN,67448_20180507,05/07/2018 12:00:00 AM,92.0,Routine - Unscheduled,67448_20180507_103149,Wiping cloths not clean or properly stored or ...,Low Risk
2,1757,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,"(37.789784, -122.420455)",NaN,1757_20170928,09/28/2017 12:00:00 AM,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
3,93022,Wise Sons Delicatessen,537 Octavia St,San Francisco,CA,94102,NaN,NaN,NaN,NaN,93022_20180907,09/07/2018 12:00:00 AM,92.0,Routine - Unscheduled,93022_20180907_103119,Inadequate and inaccessible handwashing facili...,Moderate Risk
4,4864,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,"(37.759174, -122.419066)",1.415583e+10,4864_20161206,12/06/2016 12:00:00 AM,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk


Each restaurant may have different scores from different inspections, here I'm getting the average score and group by the restaurant

In [10]:
dfs.rename(columns={'business_name': 'name', 'business_address': 'address'}, inplace=True)
dfsc=dfs[['name','inspection_score']]
dfscg=dfsc.groupby(dfs['name']).mean().round()

Join two forms together, toghether with ratings for each specific restaurant

In [11]:
dfj = pd.merge(dfc, dfscg, on='name', how='inner')

dfr=pd.DataFrame([])
for i,val in enumerate(dfj.name):
    target=val
    rownum=dataframe_filtered.index[dataframe_filtered.name==target].tolist()
    venue_id=dataframe_filtered.iloc[rownum[0],15]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    result['response']['venue']
    dfr=dfr.append({'name':val, 'rating':result['response']['venue']['rating']},ignore_index=True)
dfj2 = pd.merge(dfj, dfr, on='name', how='inner')
dfj2

,name,categories,address,distance,lat,lng,id,inspection_score,rating
0,Japanese House,Japanese Restaurant,480 6th St,1457,37.776341,-122.403028,5a3adef8ca18ea1428d66942,82.0,5.7
1,An Japanese Restaurant,Sushi Restaurant,22 Peace Plz #510,1663,37.785029,-122.429155,563ba07ccd10e452f21ab439,98.0,7.9
2,Akira Japanese Restaurant,Restaurant,1634 Bush St,1813,37.788234,-122.424170,585cab80bbec66033bb4b92f,88.0,7.9
3,Rock Japanese Cuisine,Japanese Restaurant,Pine St,2415,37.791562,-122.405772,57a16052498ef2fd3cadd5a3,85.0,5.7
4,Niji Japanese Grille,Noodle House,50 Post St,2348,37.789393,-122.402882,4a0260bbf964a5204b711fe3,86.0,5.1


Visualize the result on a map

In [12]:
restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=14) # generate map centred around the current location

# add a red circle marker to represent the current location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Your location',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6
).add_to(restaurant_map)

# add the parkinglots as blue circle markers
for lat, lng, name in zip(dfj2.lat, dfj2.lng,dfj2.name):
    popup=folium.Popup(name,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=popup,
        fill = True,
        fill_color='puple',
        fill_opacity=0.6
    ).add_to(restaurant_map)

# display map
display(restaurant_map)

After selecting a restaurant,  we can explore the parking options

In [13]:
#Locate the restaurant of choice
selection= input('Please select a restaurant from the list:').strip()
row2=dfj2.index[dfj2.name==selection].tolist()
venue_lat=dfj2.iloc[row2[0],4]
venue_lng=dfj2.iloc[row2[0],5]

#find parking lot close to the restaurant
latitude = venue_lat
longitude = venue_lng
print(latitude, longitude)
search_query = 'Parking'
radius =1000  #meters
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Please select a restaurant from the list: An Japanese Restaurant


37.785028705277405 -122.42915486228173
Parking .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=0W1MHKERIP4CAUL5LAVS4XORC2C2D51SB2TGJAHZ44BVICVD&client_secret=E1HWUVNWVVP4DXCT02QTFF0YNNIAA1M2SWU3QR5JNC1JEOE1&ll=37.785028705277405,-122.42915486228173&v=20180604&query=Parking&radius=1000&limit=50'

In [14]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# Define information and filter the dataframe: keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


dfp=dataframe_filtered[['name','categories','address','distance','lat','lng','id']]
dfp=dfp.sort_values(by=['distance'])
dfp.head()

,name,categories,address,distance,lat,lng,id
2,Japantown Parking Garage,Parking,NaN,91,37.785253,-122.430154,4de2cfe3fa7651589f08bc2d
7,B840 Japan Center Parking Lot,None,NaN,121,37.784763,-122.430492,5160ccb1e4b0ad59dab36b31
8,Webster parking,Parking,1489 Webster St,249,37.784351,-122.431858,50c600b5e4b0e9bc08e4309f
11,Fillmore Center Public Parking Garage,Parking,1345 Fillmore St,433,37.782471,-122.432870,4c8860e40f3c236a7fadf25c
6,Red’s Parking,Baseball Field,NaN,554,37.780627,-122.426194,5a47430749281473fe980852


In [15]:
#create a Map
parking_map = folium.Map(location=[latitude, longitude], zoom_start=14) 

In [16]:
# add a red circle marker to represent the current location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Restaurant location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(parking_map)

# add the parkinglots as blue circle markers
for lat, lng, name in zip(dfp.lat, dfp.lng, dfp.name):
    popup=folium.Popup(name,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=popup,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(parking_map)


# display map
display(parking_map)